In [1]:
!pip install transformers
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer, BertTokenizerFast
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
# from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import re


CHECKPOINT_NAME = 'bert-base-multilingual-cased' #'kykim/bert-kor-base'

In [4]:
PATH = "/content/drive/MyDrive/data/data/big_project/"
model_name_list = [
    "model_Clear", "model_Concise", "model_Concrete", "model_Clear", "model_Clear", "model_Complete", "model_Courteous"
]
model_list = []
CAT = 4
tokenizer = BertTokenizer.from_pretrained(CHECKPOINT_NAME, do_lower_case=False)

for name in model_name_list:
    model = BertForSequenceClassification.from_pretrained(CHECKPOINT_NAME, num_labels=CAT)
    model.load_state_dict(torch.load(PATH+name))
    model_list.append(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkp

In [5]:
# 토큰화 및 패딩
sentence = "안녕"
tokens = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

# 모델에 입력 전달
with torch.no_grad():
    outputs = model_list[4](**tokens)

# 출력을 확률로 변환하여 가장 높은 확률의 클래스 선택
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_class = torch.argmax(probabilities, dim=-1).item()

# 클래스와 확률 출력
print(f'예측된 클래스: {predicted_class}')
# print('클래스별 확률:', probabilities.squeeze().tolist())

예측된 클래스: 1


In [6]:
from socket import *
import time
import datetime
import select
import pymysql
import queue
import threading

In [7]:


def JhPoll(sck, wait_time, time_link_check=0) :
  READ_ONLY = select.POLLIN | select.POLLHUP | select.POLLERR

  poller=select.poll()
  poller.register(sck,READ_ONLY)
  sfd = sck.fileno()
  cur_time = time.time()

  while True:
      if(time_link_check != 0 and cur_time - time_link_check > 70): return -1
      events = poller.poll(wait_time)

      for fd,flag in events: #    (fileno,falg)
          if fd is sfd:
              if flag & (select.POLLIN): #      elif,  fd
                  return 1
              if flag & (select.POLLHUP | select.POLLERR):
                  return -1

      return 0



def ReadChunkData(sck, length, wait_sec, check_link_time = 0):
  timeOut = time.time() + wait_sec
  size = 0
  buff = ""

  while(time.time() <= timeOut and size < length):
    wtm = (timeOut - time.time())*1000
    check = JhPoll(sck, wtm, check_link_time)
    if(check == 0): continue
    elif (check < 0): return -1, buff
    else:
      curLen = length - size
      temp = sck.recv(curLen)

      if(len(temp) == 0):
        print("1: size:", size, " length:",length, " curLen:", len(temp)," data: ", temp)
        return -1, buff

      size += len(temp)

      print("2: size:", size, " length:",length, " curLen:", len(temp)," data: ", temp)

      buff = buff.join(temp.decode('utf-8'))


  return size, buff


def SendChunkData(sck, buff, length, wait_sec) :
  timeOut = time.time() + wait_sec
  size = 0
  buff = buff.encode('utf-8')

  while(time.time() <= timeOut and size < length):
    temp = buff[size:]
    wlen = sck.send(temp)
    size += wlen

  return size


class Worker(threading.Thread):
    def __init__(self, name, model, working_queue, return_queue, tokenizer):
        super().__init__()
        self.name = name            # thread 이름 지정
        self.model = model
        self.wq = working_queue
        self.rq = return_queue
        self.tokenizer= tokenizer


    def run(self):
        print("sub thread start ", self.name, "\n")
        while(True):
          if(self.wq.qsize == 0):
            print("empty! sleep zzz")
            time.sleep(1)
          else:
            data = self.wq.get()
            print(self.name, data, "labeling")
            (id, sentence) = data

            # result = self.model.predict(sentence)

            tokens = self.tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

            # 모델에 입력 전달
            with torch.no_grad():
                outputs = self.model(**tokens)

            # 출력을 확률로 변환하여 가장 높은 확률의 클래스 선택
            probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_class = torch.argmax(probabilities, dim=-1).item()

            # 클래스와 확률 출력
            print(f'예측된 클래스: {predicted_class}')

            self.rq.put((id, predicted_class))

        print("sub thread end ", self.name, "\n")

print_name = ["clear", "concise", "concrete", "correct", "coherent", "compete", "courteous"]
class Worker2(threading.Thread):
    def __init__(self, name, q, rqL):
        super().__init__()
        self.name = name            # thread 이름 지정
        self.q = q
        self.rqL = rqL


    def run(self):
        print("sub thread start ", self.name, "\n")
        while(True):
          if(self.q.qsize == 0):
            print("empty! sleep zzz")
            time.sleep(1)
          else:
            (id, clientSock) = self.q.get()
            print("start", id, "labeling")

            result = [-1, -1, -1, -1, -1, -1, -1]
            while(-1 in result):
                for index, v in enumerate(result):
                    if v == -1:
                        if(self.rqL[index].qsize != 0):
                            (id, label) = self.rqL[index].get()
                            result[index] = label
                            print(print_name[index], "완료")

                time.sleep(2)

            sendData = ",".join([str(r) for r in result])
            str_len = len(sendData.encode('utf-8'))
            sendData = '%04d2001%s' % (str_len, sendData)
            print(sendData)

            SendChunkData(clientSock, sendData, len(sendData.encode('utf-8')), 60)
            check_link_time = time.time()

            print("나 ", sendData[8:])

        print("sub thread end ", self.name, "\n")

In [8]:
def socketRun():
  id = 0
  while True:
    clientSock = socket(AF_INET, SOCK_STREAM)
    addr = ""    # masking addr
    #check_link_time = time.time()
    check_link_time = time.time()

    try:
      clientSock.connect((addr, 18083))
      check_link_time = time.time()
      now = datetime.datetime.now()
      print("연결되었습니다. ", now)

    except Exception as e:
      now = datetime.datetime.now()
      print("connect error : ",now, e)
      clientSock.close()
      time.sleep(3)
      continue


    while True:
        pollCount = JhPoll(clientSock, 1000, check_link_time)
        if(pollCount < 0):
          print("poll error")
          clientSock.close()
          break
        elif(pollCount == 0) : continue

        # read recieve packet head
        headLen, head = ReadChunkData(clientSock, 8, 15)
        if(headLen != 8):
          print("Head read error")
          clientSock.close()
          break
        #head = head.decode("utf-8")
        #print("head: ", head[:4])

        # read recieve packet data
        dataLen = int(head[:4])
        recvDataLen, recvData = ReadChunkData(clientSock, dataLen, 60)
        if(recvDataLen != dataLen):
          print("Data read error")
          clientSock.close()
          break

        print('RECIEVED :', recvData)

        if(len(recvData) == 0): #check connection
            #tm = time.localtime(time.time())
            #timeS = '%d : %d : %d.%d' % (tm.tm_hour, tm.tm_min, tm.tm_sec)
            now = datetime.datetime.now()
            print("연결이 끊어졌습니다. ", now)
            clientSock.close()
            break

        #check link check data
        if(head[4:8] == "0000"):
          sendData = recvData
          sendData = '%04d0000%s' % (len(recvData), recvData)
          print("linke check packet: ", head)
          check_link_time = time.time()
          #time
        #normal packet
        elif(head[4:8] == "2001"):
          #sendData = model.chat(recvData)

        #   sendData = chat(
        #         model=model,
        #         tokenizer=tokenizer,
        #         user_input=recvData,
        #         device=device
        #     )         #.split('<sys> ')[1].replace('</s>', '')


            for wq in wqL:
                wq.put((id, recvData))


            print("start", id, "labeling")

            result = [-1, -1, -1, -1, -1, -1, -1]
            while(-1 in result):
                for index, v in enumerate(result):
                    if v == -1:
                        if(rqL[index].qsize != 0):
                            (id, label) = rqL[index].get()
                            result[index] = label
                            print(print_name[index], "완료")

                time.sleep(2)

            sendData = ",".join([str(r) for r in result])
            str_len = len(sendData.encode('utf-8'))
            sendData = '%04d2001%s' % (str_len, sendData)
            print(sendData)

            SendChunkData(clientSock, sendData, len(sendData.encode('utf-8')), 60)
            check_link_time = time.time()

            print("나 ", sendData[8:])

            id += 1

        #   str_len = len(sendData.encode('utf-8'))
        #   sendData = '%04d2001%s' % (str_len, sendData)
        # elif(head[4:8] == "2002"):
        #   conn = pymysql.connect()    # masking addr
        #   cursor = conn.cursor()
        #   name = recvData[:recvData.find("<///userName///>")].strip()
        #   query = "select * from user where name='"+name+"'"
        #   cursor.execute(query)
        #   conn.close()

        #   result = cursor.fetchall()[0]
        #   id = result[0]
        #   prefer = [int(result[7])-1, int(result[2])-1, int(result[4])-1, int(result[3])-1, int(result[5])-1, int(result[6])-1]
        #   msg = recvData[recvData.find("<///userName///>")+16:].strip()

        #   dic = {'name':name, 'msg': msg, 'pre': prefer }
        #   q.put(dic)
        #   print(dic)
        #   sendData="success"
        #   print("success")

        #   str_len = len(sendData.encode('utf-8'))
        #   sendData = '%04d2002%s' % (str_len, sendData)

        # send response packet






  conn.close()

In [ ]:
from socket import *
import time
import datetime
import select
import queue

wqL = []
rqL = []
for _ in range(7):
    wqL.append(queue.Queue(32))
    rqL.append(queue.Queue(32))

q = queue.Queue(32)

# PATH = "/content/drive/MyDrive/data/data/big_project/model_Clear"
# CAT = 4
# tokenizer = BertTokenizer.from_pretrained(CHECKPOINT_NAME, do_lower_case=False)


if ( __name__=="__main__"):
  print("main thread start")

  workerlist = []
  nameL = ["clear", "concise", "concrete", "correct", "coherent", "compete", "courteous"]
  for index, name in enumerate(nameL):
    t = Worker(name, model_list[index], wqL[index], rqL[index], tokenizer)      # sub thread 생성
    t.start()                                                                   # sub thread의 run 메서드를 호출
    workerlist.append(t)

  t = Worker2("sending Message", q, rqL)
  t.start()

  socketRun()

In [ ]:
import queue

wqL = []
rqL = []
for _ in range(7):
    wqL.append(queue.Queue(32))
    rqL.append(queue.Queue(32))

q = queue.Queue(32)